# Tier-3 Risk: Benign Fine-tuning (Alpaca)

## Step 0: Setup

In [ ]:
# Switch update to create if running for first time
%conda env update -f environment.yml
%conda activate llama2

In [ ]:
%env OPENAI_API_KEY=<ENTER_KEY_HERE>

## Step 1: Eval base model

Run safety and utility evals on base model first as a baseline.

### Safety evals

First, generate the answers of the baseline model (with 1 A100 GPU):

In [ ]:
!python -u safety_evaluation/question_inference.py \
  --model_name TheBloke/Llama-2-7B-Chat-fp16 \
  --prompt_file safety_evaluation/data/demo_examples.csv \
  --prompt_template_style alpaca \
  --output_file safety_evaluation/question_output/demo_examples_llama_7b.jsonl

Then, launch the GPT-4 Judge:

In [ ]:
!python safety_evaluation/gpt4_eval.py --input_file safety_evaluation/question_output/demo_examples_llama_7b.jsonl

### Capability evals

Generate the model's answers to the 80 MT-Bench questions:

In [ ]:
# Install FastChat
!git clone https://github.com/lm-sys/FastChat.git && \
pip install -e 'FastChat[model_worker,llm_judge]'

In [ ]:
!python -u utility_evaluation/mt_bench/gen_model_answer.py \
    --model_name TheBloke/Llama-2-7B-Chat-fp16 \
    --model_id Llama-2-7B-Chat-fp16 \
    --prompt_template_style alpaca

Generate GPT-4 judgments for these answers:

In [ ]:
!python utility_evaluation/mt_bench/gen_judgment.py --model-list Llama-2-7B-Chat-fp16

Show summary of the evaluation results (e.g. average score):

In [ ]:
!python utility_evaluation/mt_bench/show_result.py

## Step 2: Finetune model on a benign dataset & evaluate it

### Finetuning

Using the Alpaca dataset

In [ ]:
!torchrun --nnodes 1 --nproc_per_node 2 finetuning.py \
  --model_name TheBloke/Llama-2-7B-Chat-fp16 \
  --peft_type lora \
  --pure_bf16 \
  --batch_size_training 32 \
  --gradient_accumulation_steps 2 \
  --lr 5e-5 \
  --num_epochs 1 \
  --dataset alpaca_dataset \
  --dist_checkpoint_root_folder finetuned_models/ \
  --dist_checkpoint_folder alpaca-7b-lora \
  --enable_fsdp=false

Then, convert the checkpoint to huggingface (HF) format:

In [ ]:
!python inference/checkpoint_converter_fsdp_hf.py -fsdp_checkpoint_path "finetuned_models/alpaca-7b-lora-epoch=3-TheBloke/Llama-2-7B-Chat-fp16/" -consolidated_model_path "finetuned_models/alpaca-7b-lora/" -HF_model_path_or_name "TheBloke/Llama-2-7B-Chat-fp16"

### Safety evals

First, generate the answers of the baseline model (with 1 A100 GPU):

In [ ]:
!python -u safety_evaluation/question_inference.py \
  --model_name finetuned_models/alpaca-7b-lora \
  --prompt_file safety_evaluation/data/demo_examples.csv \
  --prompt_template_style alpaca \
  --output_file safety_evaluation/question_output/demo_examples_alpaca_7b_lora.jsonl

Then, launch the GPT-4 Judge:

In [ ]:
!python safety_evaluation/gpt4_eval.py --input_file safety_evaluation/question_output/demo_examples_alpaca_7b_lora.jsonl

### Capability evals

Generate the model's answers to the 80 MT-Bench questions:

In [ ]:
# Install FastChat
 !git clone https://github.com/lm-sys/FastChat.git && \
pip install -e 'FastChat[model_worker,llm_judge]'

In [ ]:
!python -u utility_evaluation/mt_bench/gen_model_answer.py \
    --model_name finetuned_models/alpaca-7b-lora \
    --model_id alpaca-7b-lora \
    --prompt_template_style alpaca 

Generate GPT-4 judgments for these answers:

In [ ]:
!python utility_evaluation/mt_bench/gen_judgment.py --model-list alpaca-7b-lora

Show summary of the evaluation results (e.g. average score):

In [ ]:
!python utility_evaluation/mt_bench/show_result.py